In [ ]:
!git clone https://github.com/TRahulsingh/DeepfakeDetector.git
%cd DeepfakeDetector
!pip install -r requirements.txt

fatal: destination path 'DeepfakeDetector' already exists and is not an empty directory.
/content/DeepfakeDetector


In [ ]:
import kagglehub
path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")

Using Colab cache for faster access to the '140k-real-and-fake-faces' dataset.


In [ ]:
import kagglehub, os
path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")
print("ACTUAL PATH:", path)
print("Contents:", os.listdir(path))


Using Colab cache for faster access to the '140k-real-and-fake-faces' dataset.
ACTUAL PATH: /kaggle/input/140k-real-and-fake-faces
Contents: ['valid.csv', 'real_vs_fake', 'train.csv', 'test.csv']


In [ ]:
import os
repo_data = "/content/DeepfakeDetector/data"
folders = [
    "train/real",
    "train/fake",
    "validation/real",
    "validation/fake"
]
for f in folders:
    os.makedirs(os.path.join(repo_data, f), exist_ok=True)

print("Repo data folders ready.")


Repo data folders ready.


In [ ]:
SRC = os.path.join(path, "real_vs_fake", "real-vs-fake")
print("SRC contents:", os.listdir(SRC))

SRC contents: ['valid', 'test', 'train']


In [ ]:
# %cd /content/DeepfakeDetector
import os
base = "/content/DeepfakeDetector/data"
paths = [
    "train/real",
    "train/fake",
    "validation/real",
    "validation/fake"
]
for p in paths:
    os.makedirs(os.path.join(base, p), exist_ok=True)
print("✅ Repo directory structure ready")

✅ Repo directory structure ready


In [ ]:
DST = "/content/DeepfakeDetector/data"
import os
for p in ["train/real", "train/fake", "validation/real", "validation/fake"]:
    os.makedirs(os.path.join(DST, p), exist_ok=True)


In [ ]:
import shutil
def copy_folder(src, dst):
    for f in os.listdir(src):
        shutil.copy(os.path.join(src, f), dst)
# Train
copy_folder(os.path.join(SRC, "train", "real"), os.path.join(DST, "train", "real"))
copy_folder(os.path.join(SRC, "train", "fake"), os.path.join(DST, "train", "fake"))
# Validation
copy_folder(os.path.join(SRC, "valid", "real"), os.path.join(DST, "validation", "real"))
copy_folder(os.path.join(SRC, "valid", "fake"), os.path.join(DST, "validation", "fake"))
print("✅ Data copied successfully")


✅ Data copied successfully


In [ ]:
%cd /content/DeepfakeDetector
!pwd
!ls data/train/real | head
!ls data/train/fake | head
!ls data/validation/real | head
!ls data/validation/fake | head


/content/DeepfakeDetector
/content/DeepfakeDetector
00000.jpg
00002.jpg
00003.jpg
00006.jpg
00009.jpg
00010.jpg
00011.jpg
00012.jpg
00013.jpg
00014.jpg
001DDU0NI4.jpg
002KDWZBHU.jpg
002PMM0QG9.jpg
002TJAKUYX.jpg
003IRD4LS5.jpg
003QUB4VC9.jpg
007A5R0QAV.jpg
0086I8H5ML.jpg
009X14ED19.jpg
00A0WLZE5X.jpg
00005.jpg
00008.jpg
00020.jpg
00024.jpg
00026.jpg
00034.jpg
00037.jpg
00039.jpg
00041.jpg
00047.jpg
00483R5CC4.jpg
007SMMOPYB.jpg
008Y48BIX8.jpg
00BYVMEI3I.jpg
00GN2JWV43.jpg
00JMWC4S8R.jpg
00V8VPXL3V.jpg
00W5NPIX4S.jpg
012N2T4FKG.jpg
013O4R3GO3.jpg


In [ ]:
!find data/validation/real -type f | tail -n +3001 | xargs rm -f
!find data/validation/fake -type f | tail -n +3001 | xargs rm -f


In [ ]:
!touch datasets/__init__.py


In [38]:
!ls datasets


hybrid_loader.py  __init__.py


In [39]:
# !ls data/train/real | wc -l
!ls data/train/fake | wc -l
!ls data/validation/real | wc -l
!ls data/validation/fake | wc -l


3000
3000
3000


In [ ]:
%cd /content/DeepfakeDetector
!python main_trainer.py


Streaming output truncated to the last 5000 lines.
                                    0:00:00                    val_loss: 0.113  
                                                               val_acc: 0.966   
Validation ━━━━━━━━━━━━━╺━━ 630/750 0:00:29 •        21.50it/s                  
Epoch 1/2  ━━━━━━━━━━━━━━━━ 750/750 0:00:59 •        12.49it/s v_num: 8.000     
                                    0:00:00                    val_loss: 0.113  
                                                               val_acc: 0.966   
Validation ━━━━━━━━━━━━━╺━━ 631/750 0:00:29 •        21.49it/s                  
Epoch 1/2  ━━━━━━━━━━━━━━━━ 750/750 0:00:59 •        12.49it/s v_num: 8.000     
                                    0:00:00                    val_loss: 0.113  
                                                               val_acc: 0.966   
Validation ━━━━━━━━━━━━━╺━━ 632/750 0:00:29 •        21.50it/s                  
Epoch 1/2  ━━━━━━━━━━━━━━━━ 750/750 0:00:59 •        12.49

In [40]:
!ls models


best_model.ckpt     best_model-v2.ckpt	best_model-v3.pt
best_model-v1.ckpt  best_model-v3.ckpt	best_model-v4.ckpt


In [42]:
import torch
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from lightning_modules.detector import DeepfakeDetector
from PIL import Image
import os

# 1. Rebuild the SAME backbone as training
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
backbone = efficientnet_b0(weights=weights)

in_features = backbone.classifier[1].in_features
backbone.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.4),
    torch.nn.Linear(in_features, 2)   # IMPORTANT: 2 classes
)

# 2. Load checkpoint
model = DeepfakeDetector.load_from_checkpoint(
    "models/best_model.ckpt",
    model=backbone,
    lr=0.0001
)

model.eval()
model.cuda()


DeepfakeDetector(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [43]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

img_path = "data/validation/real/" + os.listdir("data/validation/real")[0]
img = Image.open(img_path).convert("RGB")

x = transform(img).unsqueeze(0).cuda()

with torch.no_grad():
    logits = model(x)
    probs = torch.softmax(logits, dim=1)

print("Fake probability:", probs[0][0].item())
print("Real probability:", probs[0][1].item())


Fake probability: 0.02428715117275715
Real probability: 0.9757128357887268


In [46]:
import gradio as gr
import torch
import mimetypes
import cv2
import os

from PIL import Image
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

from lightning_modules.detector import DeepfakeDetector

# ===============================
# Load Model (CORRECT WAY)
# ===============================
def load_model():
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    backbone = efficientnet_b0(weights=weights)

    in_features = backbone.classifier[1].in_features
    backbone.classifier = torch.nn.Sequential(
        torch.nn.Dropout(0.4),
        torch.nn.Linear(in_features, 2)
    )

    model = DeepfakeDetector.load_from_checkpoint(
        "models/best_model.ckpt",
        model=backbone,
        lr=0.0001,
        map_location="cpu"
    )

    model.eval()
    return model

model = load_model()

# ===============================
# Preprocessing
# ===============================
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ===============================
# Prediction Logic
# ===============================
def predict_file(file_obj):
    if file_obj is None:
        return "⚠️ No file selected", "", None

    path = file_obj.name
    mime, _ = mimetypes.guess_type(path)

    # ---------- IMAGE ----------
    if mime and mime.startswith("image"):
        img = Image.open(path).convert("RGB")
        x = preprocess(img).unsqueeze(0)

        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[0]

        fake_prob = probs[0].item()
        real_prob = probs[1].item()

        label = "🔴 Deepfake" if fake_prob > real_prob else "🟢 Real"
        confidence = max(fake_prob, real_prob) * 100

        return label, f"{confidence:.2f}%", img

    # ---------- VIDEO ----------
    elif mime and mime.startswith("video"):
        cap = cv2.VideoCapture(path)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            return "❌ Error reading video", "", None

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        x = preprocess(img).unsqueeze(0)

        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[0]

        fake_prob = probs[0].item()
        real_prob = probs[1].item()

        label = "🔴 Deepfake (sampled frame)" if fake_prob > real_prob else "🟢 Real (sampled frame)"
        confidence = max(fake_prob, real_prob) * 100

        return label, f"{confidence:.2f}%", img

    else:
        return "❌ Unsupported file type", "", None

# ===============================
# Gradio UI
# ===============================
with gr.Blocks(title="Deepfake Detector") as demo:
    gr.Markdown(
        "## 🧠 Deepfake Detector\n"
        "Upload an **image or video**. For videos, the model analyzes a sampled frame."
    )

    file_input = gr.File(
        label="Upload Image or Video",
        file_types=[".jpg", ".jpeg", ".png", ".mp4", ".mov"]
    )

    with gr.Row():
        prediction = gr.Textbox(label="Prediction")
        confidence = gr.Textbox(label="Confidence")

    preview = gr.Image(label="Preview")

    file_input.change(
        fn=predict_file,
        inputs=file_input,
        outputs=[prediction, confidence, preview]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a9fd8ecceae313f0a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [57]:
import gradio as gr
import torch
import mimetypes
import cv2
import os

from PIL import Image
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

from lightning_modules.detector import DeepfakeDetector

# ===============================
# Face Detector (LIGHTWEIGHT)
# ===============================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ===============================
# Load Model (CORRECT WAY)
# ===============================
def load_model():
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    backbone = efficientnet_b0(weights=weights)

    in_features = backbone.classifier[1].in_features
    backbone.classifier = torch.nn.Sequential(
        torch.nn.Dropout(0.4),
        torch.nn.Linear(in_features, 2)
    )

    model = DeepfakeDetector.load_from_checkpoint(
        "models/best_model.ckpt",
        model=backbone,
        lr=0.0001,
        map_location="cpu"
    )

    model.eval()
    return model

model = load_model()

# ===============================
# Preprocessing
# ===============================
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ===============================
# Extract FACE-CROPPED Frames
# ===============================
def extract_face_frames(video_path, max_frames=200):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS) or 25)

    faces_out = []
    frame_count = 0

    while len(faces_out) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % fps == 0:  # ~1 frame per second
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            if len(faces) > 0:
                x, y, w, h = faces[0]  # take first detected face
                face = frame[y:y+h, x:x+w]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                faces_out.append(Image.fromarray(face))

        frame_count += 1

    cap.release()
    return faces_out

# ===============================
# Predict VIDEO
# ===============================
def predict_video(video_path):
    faces = extract_face_frames(video_path)

    if len(faces) == 0:
        return "❌ No face detected in video", "", None

    fake_probs = []

    with torch.no_grad():
        for img in faces:
            x = preprocess(img).unsqueeze(0)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[0]
            fake_probs.append(probs[0].item())

    avg_fake = sum(fake_probs) / len(fake_probs)
    max_fake = max(fake_probs)

    final_fake = 0.6 * avg_fake + 0.4 * max_fake
    label = "🔴 Deepfake" if final_fake > 0.5 else "🟢 Real"
    confidence = max(final_fake, 1 - final_fake) * 100

    return label, f"{confidence:.2f}%", faces[0]

# ===============================
# Predict IMAGE
# ===============================
def predict_image(image_path):
    img = Image.open(image_path).convert("RGB")
    x = preprocess(img).unsqueeze(0)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)[0]

    fake_prob = probs[0].item()
    real_prob = probs[1].item()

    label = "🔴 Deepfake" if fake_prob > real_prob else "🟢 Real"
    confidence = max(fake_prob, real_prob) * 100

    return label, f"{confidence:.2f}%", img

# ===============================
# Unified Predictor
# ===============================
def predict_file(file_obj):
    if file_obj is None:
        return "⚠️ No file selected", "", None

    path = file_obj.name
    mime, _ = mimetypes.guess_type(path)

    if mime and mime.startswith("image"):
        return predict_image(path)

    elif mime and mime.startswith("video"):
        return predict_video(path)

    else:
        return "❌ Unsupported file type", "", None

# ===============================
# Gradio UI
# ===============================
with gr.Blocks(title="Deepfake Detector") as demo:
    gr.Markdown(
        "## 🧠 Deepfake Detector\n"
        "Upload an **image or video**.\n\n"
        "- Images are analyzed directly\n"
        "- Videos are analyzed by sampling **multiple face frames**"
    )

    file_input = gr.File(
        label="Upload Image or Video",
        file_types=[".jpg", ".jpeg", ".png", ".mp4", ".mov"]
    )

    with gr.Row():
        prediction = gr.Textbox(label="Prediction")
        confidence = gr.Textbox(label="Confidence")

    preview = gr.Image(label="Preview (Sampled Frame / Image)")

    file_input.change(
        fn=predict_file,
        inputs=file_input,
        outputs=[prediction, confidence, preview]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6db298af720135ddc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
